In [18]:
from enum import Enum
import numpy as np


class State(Enum):
    NEW = 0
    CLOSED = 1
    OPEN = 2


class Point:

    def __init__(self, x, y):

        self.x = x
        self.y = y

    def __eq__(self, other):
        if not isinstance(other, Point):
            return False
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))

    def __str__(self):
        return f'Point({self.x}, {self.y})'
    
    def __repr__(self):
        return self.__str__()
    
    def distance(self, other):
        return np.sqrt((self.x - other.x) ** 2 + (self.y - other.y) ** 2)


class Node:

    def __init__(self, point, state=State.NEW, k=0.0, h=float('inf'), parent=None):
        self.point = point
        self.state = state

        # The key value is used to prioritize states for exploration 
        # and determines their importance in the search process.
        self.k = k
        self.h = h
        self.parent = parent

    def __eq__(self, other):
        if not isinstance(other, Node):
            return False
        return self.point == other.point and self.state == other.state

    def __str__(self):
        chr = '   '
        if self.state == State.OPEN:
            chr = ' O '
        elif self.state == State.CLOSED:
            chr = ' X '
        return f'{chr}'

    def __repr__(self):
        return f'Node({self.point}, state={self.state.name})'

    def __hash__(self):
        return hash(self.point)
    
    def distance(self, other):
        return self.point.distance(other.point)


class DStar:

    def __init__(self, width, height, start_point, goal_point):
    
        self.width = width
        self.height = height
        self.start_point = start_point
        self.goal_point = goal_point

        self.map_bounds = (0, width, 0, height)

        # My other implementations dynamically create nodes from a continuous map
        # for search based algorithms. For this algorithm I'll try to keep things
        # simple as the overally complexity is much higher than other algorithms
        self.grid = None
        self.start_node = None

        # Contains nodes that are candidates for expansion. 
        # These are nodes that have been discovered but not yet fully explored
        self.open_set = None
        
        # The visited set (sometimes referred to as the closed set in other algorithms 
        # like A*) contains nodes that have been fully explored
        self.visited_set = None
        
        self.obs_set = None  # Assuming obstacles are stored in a set of Point objects
        
        self.path = None

    def __str__(self):
        string_repr = '-' * 100
        for i in range(self.width):
            for j in range(self.height):
                chr = 'N'
                if self.grid[i][j].state == State.OPEN:
                    chr = 'O'
                elif self.grid[i][j].state == State.CLOSED:
                    chr = 'C'
                string_repr += f' {chr} '
            string_repr += '\n'
        return string_repr 

    def init(self):

        self.path = []

        # Initialize the grid
        self.grid = [[Node(Point(x, y)) for y in range(self.height)] for x in range(self.width)]
        self.start_node = self.grid[self.start_point.x][self.start_point.y]

        # Initialize the sets
        self.open_set = set()
        self.visited_set = set()
        self.obs_set = set()

        # Set goal node's h value to 0
        goal_node = self.grid[self.goal_point.x][self.goal_point.y]
        goal_node.h = 0

        # Insert goal node into open set
        self.insert(goal_node, 0)

    def check_collision(self, node_1, node_2):
        return False

    def pre_search(self):
        self.init()

    def can_run(self):
        # Continue processing nodes until the start state is marked as 'CLOSED'
        return self.grid[self.start_point.x][self.start_point.y].state != State.CLOSED

    def post_search(self):
        self.extract_path()

    def extract_path(self):

        current_point = self.start_node
        while current_point is not None:
            self.path.append(current_point.point)
            print(f'Appending {repr(current_point.point)} with parent {current_point.parent} to the path')
            current_point = current_point.parent
        # self.path.reverse()

    def extract_path_legacy(self):

        current_point = self.goal_point
        while current_point != self.start_point:
            self.path.append(current_point)
            current_node = self.grid[current_point.x][current_point.y]
            current_point = current_node.parent.point
        self.path.append(self.start_point)
        self.path.reverse()
    
    def min_state(self):
        """
        Select the node from the open set with the minimum k value
        """

        if not self.open_set:
            return None
        return min(self.open_set, key=lambda x: x.k)
    
    def delete(self, node):
        """
        Remove a node from the open set
        """

        if node.state == State.OPEN:
            node.state = State.CLOSED
        self.open_set.remove(node)

    def insert(self, node, new_h):
        """
        Insert the node into the open set
        """

        if node.state == State.NEW:
            node.k = new_h
        elif node.state == State.OPEN:
            node.k = min(node.k, new_h)
        elif node.state == State.CLOSED:
            node.k = min(node.h, new_h)
            
        node.h = new_h
        node.state = State.OPEN
        self.open_set.add(node)

    def get_k_min(self):
        """
        Computes the minimum value of the cost estimate (k) 
        among all nodes in the open set
        """
        
        if not self.open_set:
            return -1
        return min([x.k for x in self.open_set])
    
    def get_neighbor(self, node):
        """
        Returns a list of neighboring nodes of a given state node 
        on the grid
        """

        neighbors = set()
        for i in range(-1, 2):
            for j in range(-1, 2):
                x = node.point.x + i
                y = node.point.y + j
                if (self.map_bounds[0] <= x < self.map_bounds[1] and 
                    self.map_bounds[2] <= y < self.map_bounds[3] and
                    (x, y) not in self.obs_set):
                    neighbor_node = self.grid[x][y]
                    neighbors.add(neighbor_node)
        return neighbors

    def cost(self, node_1, node_2):
        if self.check_collision(node_1, node_2):
            return float("inf")
        return node_1.distance(node_2)

    def step_search(self):

        # Step 1: Select a node from the open set with the minimum k value
        s = self.min_state()

        # Add the selected node to the visited set
        self.visited_set.add(s)

        # If there are no states in the open set or if the selected state is None, return -1
        if s is None:
            return -1
        
        # Record the minimum k value of this iteration (min path cost)
        k_old = self.get_k_min()

        # Step 2: Mark the selected node as visited (state = CLOSED)
        self.delete(s)
                
        # Step 3: Update the costs (h) of neighboring nodes based on the current node
        if k_old < s.h:
            for neighbor in self.get_neighbor(s):
                if (neighbor.h <= k_old and
                    s.h > neighbor.h + self.cost(neighbor, s)):
                    s.parent = neighbor
                    s.h = neighbor.h + self.cost(neighbor, s)
                
        # Step 4: Update nodes if the minimum path cost (k) changes
        if k_old == s.h:
            for neighbor in self.get_neighbor(s):
                if (neighbor.state == State.NEW or 
                    (neighbor.parent == s and neighbor.h != s.h + self.cost(s, neighbor)) or
                    (neighbor.parent != s and neighbor.h > s.h + self.cost(s, neighbor))):
                    
                    # Update h value and insert the node into the open set
                    neighbor.parent = s
                    self.insert(neighbor, s.h + self.cost(s, neighbor))

        else:
            for neighbor in self.get_neighbor(s):
                if (neighbor.state == State.NEW or 
                    (neighbor.parent == s and neighbor.h != s.h + self.cost(s, neighbor))):

                    # Update h value and insert the state into the open set
                    neighbor.parent = s
                    self.insert(neighbor, s.h + self.cost(s, neighbor))
                else:
                    if (neighbor.parent != s and 
                            neighbor.h > s.h + self.cost(s, neighbor)):
                        
                        # Insert s into the open set again for further exploration
                        self.insert(s, s.h)

                    elif (neighbor.parent != s and
                            s.h > neighbor.h + self.cost(neighbor, s) and
                            neighbor.state == State.CLOSED and
                            neighbor.k > k_old):
                        
                        # Insert neighbor into the open set again for further exploration
                        self.insert(neighbor, neighbor.h)

        return self.get_k_min()

start = Point(5, 5)
goal = Point(45, 25)

dstar = DStar(50, 50, start, goal)
 
dstar.pre_search()
while dstar.can_run():
    dstar.step_search()
dstar.post_search()
print(dstar.path)

Appending Point(5, 5) with parent  X  to the path
Appending Point(6, 6) with parent  X  to the path
Appending Point(7, 7) with parent  X  to the path
Appending Point(8, 8) with parent  X  to the path
Appending Point(9, 9) with parent  X  to the path
Appending Point(10, 10) with parent  X  to the path
Appending Point(11, 11) with parent  X  to the path
Appending Point(12, 12) with parent  X  to the path
Appending Point(13, 13) with parent  X  to the path
Appending Point(14, 14) with parent  X  to the path
Appending Point(15, 15) with parent  X  to the path
Appending Point(16, 16) with parent  X  to the path
Appending Point(17, 17) with parent  X  to the path
Appending Point(18, 18) with parent  X  to the path
Appending Point(19, 19) with parent  X  to the path
Appending Point(20, 20) with parent  X  to the path
Appending Point(21, 21) with parent  X  to the path
Appending Point(22, 22) with parent  X  to the path
Appending Point(23, 22) with parent  X  to the path
Appending Point(24, 22